# HeavyDB Runtime UDF support

The *Remote Backend Compiler* (RBC) package implements the HeavyDB client support for defining so-called Runtime UDFs. That is, while HeavyDB server is running, one can register new SQL functions to HeavyDB Calcite server as well as provide their implementations in LLVM IR string form. The RBC package supports creating Runtime UDFs from Python functions.

First, we need to connect RBC to HeavyDB server:

In [8]:
import warnings; warnings.filterwarnings('ignore')

In [9]:
# NBVAL_IGNORE_OUTPUT
from rbc.heavydb import RemoteHeavyDB
heavydb = RemoteHeavyDB(user='admin', password='HyperInteractive',
                        host='127.0.0.1', port=6274)

In [10]:
import ibis
ibis_con = ibis.heavyai.connect(user=heavydb.user, password=heavydb.password,
                                host=heavydb.host, port=heavydb.port, database=heavydb.dbname)

One can use the Runtime UDF functions in SQL queries from several HeavyDB clients, for instance, from `pymapd`, `ibis.omniscidb`, or `rbc.heavydb`. In this demo, we'll use `ibis.omniscidb`. For that, we connect `ibis.omniscidb` to HeavyDB server as well:

### Create and fill test table

In [14]:
heavydb.sql_execute('drop table if exists mytable')
heavydb.sql_execute('create table if not exists mytable (x DOUBLE, i INT);')
for _i in range(5):
    heavydb.sql_execute(f'insert into mytable values {(_i, _i + 0.0)}')
ibis_con.sql('select x, i from mytable').execute()

,x,i
0,0.0,0
1,1.0,1
2,2.0,2
3,3.0,3
4,4.0,4


## Defining Runtime UDFs from Python

To define a Runtime UDF from a Python function, the function must be decorated with `omni` decorator:

In [15]:
@heavydb('f32(f32)', 'f64(f64)')
def incr(v):
    """Increment float value by one"""
    return v + 1

One can overload existing UDFs (now using Python annotations):

In [16]:
@heavydb
def incr(v: 'int32') -> 'int32':
    """Increment integer value by ten"""
    return v + 10

Finally, we register the defined UDFs to HeavyDB server:

In [17]:
heavydb.register()

## Using UDFs in a SQL query

In [18]:
t = ibis_con.sql('select x, incr(x), i, incr(i) from mytable')
t[t.i < 3].execute()

,x,EXPR$1,i,EXPR$3
0,0.0,1.0,0,10
1,1.0,2.0,1,11
2,2.0,3.0,2,12
